# Test modification #2

In [2]:
from optimize import *


Starting MATLAB engine...
-------------------------
Elapsed: 3.02 sec


In [ ]:
optimize_multiple(0, 10, seeds=[0], functions={'forest': forest_minimize}, verb=True)

Seed: 0
forest:
Iteration No: 1 started. Evaluating function at random point.
0.0001
0.001
0.0
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 70.5913
Function value obtained: 0.6332
Current minimum: 0.6332
Iteration No: 2 started. Evaluating function at random point.
0.0001
0.001
0.0


# Optimization stuff

In [1]:
from optimize import *


Starting MATLAB engine...
-------------------------
Elapsed: 19.48 sec


In [ ]:
optimize_multiple(3, 50, verb=True)

In [ ]:
reload_multiple(3, 50, 3, verb=True)

# See predicted function

In [ ]:
import matlab.engine
from optimize import *
from skopt.acquisition import _gaussian_acquisition
loaded = load("./optims/scenario6/forest_3_50.opt")

In [ ]:
# how optimizer predicts next value:
space = loaded['space']
#X = space.transform(space.rvs(n_samples=result.specs['args']['n_points'], random_state=0))
X = np.array([[10**(i/5-1), 10**(j/10+1)] for i in range(21) for j in range(14)])
values = _gaussian_acquisition(X=X, model=loaded.models[-1], y_opt=np.min(loaded.func_vals),
                               acq_func=loaded.specs['args']['acq_func'],
                               acq_func_kwargs={"xi":loaded.specs['args']['xi'], "kappa":loaded.specs['args']['kappa']})

In [ ]:
# how the actual function works
out = np.zeros((27, 41))
for i in range(27):
    maxIter = 10**(i/20.0+1)
    for j in range(41):
        alpha = 10**(j/20.0-1)
        out[i][j] = loaded.models[-1].predict([[alpha, maxIter]])[0]

savemat('./hyper_opt.mat', mdict={'surface':out, 'x':loaded.x_iters, 'y':loaded.func_vals})